In [1]:
# had help 
# from https://towardsdatascience.com/the-k-prototype-as-clustering-algorithm-for-mixed-data-type-categorical-and-numerical-fe7c50538ebb
# and https://latrobe.libguides.com/sensitivedata/deidentification
# and https://dev.to/r0f1/a-simple-way-to-anonymize-data-with-python-and-pandas-79g
# and https://github.com/qiyuangong/Mondrian/blob/master/anonymizer.py
# and https://sdcpractice.readthedocs.io/en/latest/anon_methods.html
# and https://medium.com/brillio-data-science/a-brief-overview-of-k-anonymity-using-clustering-in-python-84203012bdea

import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
from sklearn.cluster import KMeans
from kmodes.kprototypes import KPrototypes
# from plotnine import *
# import plotnine


In [2]:
df = pd.read_csv('data/2018_Central_Park_Squirrel_Census_-_Squirrel_Data.csv'
)
quasi_ids = [
    'Hectare', 'Date', 'Age', 'Primary Fur Color',
    'Highlight Fur Color', 'Location'
]

In [3]:
#helpers
all_fields = list(df)

fields_list = list(df)
del fields_list[0]

In [4]:
#establish a set of category fields 
#this will cast nulls to -1 and move strings to ints
quasi_ids_cat = list()

for c in list(df[quasi_ids]):
    df[c+'_cat'] = df[c].astype('category')
    df[c+'_cat'] = df[c+'_cat'].cat.codes
    df[c+'_cat'] = pd.to_numeric(df[c+'_cat'],errors = 'coerce')
    quasi_ids_cat.append(c+'_cat')


In [5]:
categorical = []
continuous = []

for c in quasi_ids_cat:
    col = df[c]
    nunique = col.nunique()
    if nunique < 20: # how hacky is this?
        categorical.append(c)
    else:
        continuous.append(c)


In [6]:
#just the data we need for kmeans/kmode
clustering_df = df[quasi_ids_cat].copy()

# Get the position of categorical columns
catColumnsPos = [clustering_df.columns.get_loc(col) for col in categorical]
dfMatrix = clustering_df.to_numpy()


In [7]:
#I'm sure there's a programmatic way to get to the elbow graph, but it's beyond me at the moment, so I'm picking 4 clusters

kprototype = KPrototypes(n_jobs = -1, n_clusters = 4, init = 'Huang', random_state = 0) 

kprototype.fit_predict(dfMatrix, categorical = catColumnsPos)
df['cluster_label'] = kprototype.labels_


In [8]:
#get the mode of each column by cluster
quasi_ids_mode = list()

for c in list(df[quasi_ids]):
    # this is an insane syntax just to get a windowed mode. I can't believe it's not a param in .transform()
    df[c+'_mode'] = df['cluster_label'].map(df.groupby('cluster_label')[c].agg(lambda x: x.value_counts().idxmax())) 
    quasi_ids_mode.append(c+'_mode')



In [9]:
#establish clean columns 
quasi_ids_clean = list()

for c in list(df[quasi_ids]):
    # this is an insane syntax just to get a windowed mode. I can't believe it's not a param in .transform()
    # df[c+'_clean'] = df[c].combine_first(df[c+'_cat'])
    df[c+'_clean'] = df[c].combine_first('-')
    quasi_ids_clean.append(c+'_clean')

df['n_count_of_qid']=df.groupby(quasi_ids_clean)['id'].transform('count')
df['risky_grain'] = df['n_count_of_qid'].apply(lambda x: True if x < 4 else False) #4 is a complete guess here, not sure what is standard for the industry. 


In [10]:
# #sort in desc order of unique values in the modes 
# #help from https://www.pythoncentral.io/how-to-sort-a-list-tuple-or-object-with-sorted-in-python/

def getKey(item):
  return item[1]

unique_val_arr = []

for c in quasi_ids_mode:
  unique_val_arr.append([c,df.groupby(c).size().sort_values(ascending=False).count()])

unique_val_arr = sorted(unique_val_arr, key=getKey,reverse=True)

#iterate over modes, largest to smallest
#replace low-grain rows with mode
#recount and break if we're good, if not, go through all of the qids
for arr_val in unique_val_arr:
  c = arr_val[0]
  df[c.replace("_mode", "_clean")] = np.where(df["risky_grain"] == True, df[c], df[c.replace("_mode", "_clean")])
  #recount
  df['n_count_of_qid']=df.groupby(quasi_ids_clean)['id'].transform('count')
  df['risky_grain'] = df['n_count_of_qid'].apply(lambda x: True if x < 4 else False) #4 is a complete guess here, not sure what is standard for the industry. 

  try: 
    df.groupby('risky_grain').size()[True]
  except:
    break


In [11]:
#drop rows that we couldn't fix
df.drop(df[df['risky_grain'] == True].index, inplace = True)

In [12]:
fields_for_cleaned_df = list(set(all_fields) - set(quasi_ids)) + quasi_ids_clean
cleaned_df = df[fields_for_cleaned_df].copy()

In [13]:
for c in quasi_ids_clean:
  cleaned_df.rename({c: c.replace("_clean", "")}, axis=1, inplace=True)

cleaned_df

,id,Specific Location,Indifferent,Climbing,Quaas,Tail flags,Approaches,Kuks,Other Interactions,Tail twitches,...,Chasing,Other Activities,Foraging,Running,Hectare,Date,Age,Primary Fur Color,Highlight Fur Color,Location
0,0,NaN,False,False,False,False,False,False,NaN,False,...,False,NaN,False,False,36I,10132018,-1,-1,-1,-1
1,1,NaN,False,False,False,False,False,False,NaN,False,...,False,NaN,False,False,32E,10072018,Adult,Gray,-1,-1
2,2,NaN,False,False,False,False,False,False,NaN,False,...,True,NaN,False,False,14D,10072018,-1,Gray,-1,Above Ground
3,3,NaN,False,False,False,False,False,False,NaN,False,...,False,NaN,True,False,32E,10072018,Adult,Gray,-1,-1
4,4,on tree stump,False,False,False,False,False,False,NaN,False,...,False,NaN,True,False,14D,10172018,Adult,Gray,Cinnamon,Above Ground
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3018,3018,NaN,False,False,False,False,False,False,NaN,False,...,False,NaN,True,False,30B,10072018,Adult,Gray,-1,Ground Plane
3019,3019,NaN,True,False,False,False,False,False,NaN,False,...,False,NaN,True,False,19A,10132018,Adult,Gray,White,Ground Plane
3020,3020,NaN,True,False,False,False,False,False,NaN,False,...,False,NaN,True,False,32E,10122018,Adult,Gray,"Black, Cinnamon, White",Ground Plane
3021,3021,NaN,True,False,False,False,False,False,NaN,False,...,False,NaN,False,False,32E,10072018,Adult,Gray,"Cinnamon, White",Ground Plane
